In [2]:
suppressWarnings({
    suppressPackageStartupMessages({
        library(dplyr, quietly = T)
        library(org.Mm.eg.db, quietly = T)
        library(data.table, quietly = T)
    })
})

In [ ]:
# https://medium.com/computational-biology/gene-id-mapping-using-r-14ff50eec9ba

# map MGI gene symbol to mappere gene ID (needed for CellFie)
from = 'SYMBOL'
to = 'ENTREZID'

gene.IDs<-rownames(expr)
print(paste0('The original gene list containts ', length(gene.IDs),' genes'))
# deal with one to many mappings by only retaining first
mapper<-mapIds(org.Mm.eg.db, keys=gene.IDs, keytype=from, column=to, multiVals='first')
mapper<-data.frame(mapper)
colnames(mapper)<-c('new.ID')
mapper[['original.ID']]<-rownames(mapper)

# drop unmapped genes
mapper<-mapper[!(is.na(mapper[['new.ID']])),]
expr<-expr[rownames(expr) %in% mapper$original.ID,]
print(paste0(dim(mapper)[[1]], ' genes remain after removing unmapped IDs'))

# if no many to one mappings (may be the case for org.Mm.eg.db)
rownames(expr)<-unname(setNames(mapper$new.ID, mapper$original.ID)[rownames(expr)])

# deal with many to one mappings by taking row average
expr[['new.ID']]<-unname(setNames(mapper$new.ID, mapper$original.ID)[rownames(expr)])
expr<-setDT(expr)[ , lapply(.SD, mean, na.rm=TRUE), by = new.ID, 
                     .SDcols = colnames(expr)[colnames(expr) != 'new.ID']]
print(paste0(dim(expr)[[1]], ' genes remain after mergin many-to-one mappings'))
expr<-as.data.frame(expr)
rownames(expr)<-expr[['new.ID']]
expr<-expr[, colnames(expr) != 'new.ID']